In [1]:
from pdrtpy.modelset import ModelSet
import numpy as np
from astropy.table import Table
import pdrtpy.pdrutils as utils

/home/mpound/anaconda3/lib/python3.7/site-packages/astropy/table/column.py:1020: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = getattr(super(), op)(other)


In [2]:
m = ModelSet(name="wk2006",z=1)

In [3]:
m.table.show_in_notebook()

idx,numerator,denominator,ratio,filename,z,title
,,,,,,null
0,OI_145,OI_63,OI_145/OI_63,oioiweb,1.0,[O I] 145 $\mu$m / [O I] 63 $\mu$m
1,OI_145,CII_158,OI_145/CII_158,o145ciiweb,1.0,[O I] 145 $\mu$m / [C II] 158 $\mu$m
2,OI_63,CII_158,OI_63/CII_158,oicpweb,1.0,[O I] 63 $\mu$m / [C II] 158 $\mu$m
3,CII_158,CI_609,CII_158/CI_609,ciici609web,1.0,[C II] 158 $\mu$m / [C I] 609 $\mu$m
4,CI_370,CI_609,CI_370/CI_609,ciciweb,1.0,[C I] 370 $\mu$m / [C I] 609 $\mu$m
5,CII_158,CO_10,CII_158/CO_10,ciicoweb,1.0,[C II] 158 $\mu$m / CO(J=1-0)
6,CI_609,CO_10,CI_609/CO_10,cicoweb,1.0,[C I] 609 $\mu$m / CO(J=1-0)
7,CI_609,CO_21,CI_609/CO_21,cico21web,1.0,[C I] 609 $\mu$m / CO(J=2-1)
8,CI_609,CO_32,CI_609/CO_32,cico32web,1.0,[C I] 609 $\mu$m / CO(J=3-2)


In [9]:
try:
    m.model_ratios(["CII_158"])
except Exception as e:
    print("caught exception properly:",e)

caught exception properly: m most contain at least two strings


In [10]:
for k in m.find_files(["CI_609"]):
    print(k)

In [11]:
utils.model_dir()

'/home/mpound/src/pbranch/pdrtpy/pdrtpy/models/'

In [12]:
x=m.get_models(["CI_609","FIR"],model_type="ratio")
y=m.get_models(["CI_609","FIR"],model_type="intensity")
z=m.get_models(["CI_609","FIR"],model_type="both")

In [17]:
y

{'CI_609': [[1.68704e-07 2.35549e-07 3.39000e-07 4.74282e-07 6.33437e-07 7.97366e-07
   9.35831e-07 1.02814e-06 1.05679e-06 1.03928e-06 9.88352e-07 9.16733e-07
   8.32380e-07 7.25747e-07 6.11155e-07 4.74282e-07 3.19901e-07 1.95761e-07
   1.10931e-07 6.14338e-08 3.42183e-08 1.90986e-08 1.16820e-08 6.03197e-09
   7.35296e-09]
  [1.81437e-07 2.57831e-07 3.72423e-07 5.31578e-07 7.27338e-07 9.40606e-07
   1.14273e-06 1.29552e-06 1.37669e-06 1.38465e-06 1.33849e-06 1.25732e-06
   1.14751e-06 1.02655e-06 8.76944e-07 7.14606e-07 5.09296e-07 3.29451e-07
   1.89394e-07 1.03451e-07 5.60225e-08 3.05578e-08 1.68704e-08 1.21594e-08
   1.01063e-08]
  [1.94169e-07 2.76930e-07 4.04254e-07 5.82507e-07 8.10099e-07 1.06952e-06
   1.33213e-06 1.55176e-06 1.68704e-06 1.73479e-06 1.70296e-06 1.60746e-06
   1.48173e-06 1.31939e-06 1.15069e-06 9.50155e-07 7.38479e-07 4.94972e-07
   2.99211e-07 1.63930e-07 8.67394e-08 4.56775e-08 2.43507e-08 2.45099e-08
   1.38783e-08]
  [2.06901e-07 2.97620e-07 4.36085e-07 6.3

In [ ]:
m.supported_lines["intensity label"]

In [ ]:
 set(m._supported_lines["intensity label"])

In [ ]:
t = ['CI_609','OI_63','FIR','FEII_26']

In [ ]:
a=list(set(t)&set(m._supported_lines["intensity label"]))

In [ ]:
a

In [ ]:
b=['H200S2','MASAS']

In [ ]:
a.append(b)

In [ ]:
a

In [ ]:
a.extend(b)

In [ ]:
a

In [ ]:
m._table